In [ ]:
#REQUEST dataset and target TO THE USER -- dataset in upload


"""
import tkinter
import tkinter.filedialog


#import os
#import matplotlib as mpl
#if os.environ.get('DISPLAY','') == '':
#   print('no display found. Using non-interactive Agg backend')
#   mpl.use('Agg')


print('Hello!\nThis application is designed to help you to discover, starting from a dataset (a set of columns with data) and a target variable to predict, which are the most significant'+
      ' variable determining the target ones.\n Let\'s start!!\n ')


root = tkinter.Tk()
#root.mainloop()
root.withdraw() # Remove background screen

# Ask for name of inputfiles
excelfile = tkinter.filedialog.askopenfilename(parent=root, title="Select first Excel file", defaultextension = ".xlsx")


target_variable=input('Please specify the name of the target variable present in the dataset:\n')



"""

no display found. Using non-interactive Agg backend
Hello!
This application is designed to help you to discover, starting from a dataset (a set of columns with data) and a target variable to predict, which are the most significant variable determining the target ones.
 Let's start!!
 


TclError: ignored

##USER INPUT AND INPUT VALIDATION

In [1]:


#CHECK THE EXISTENCE OF THE TARGET VARIABLE IN THE DATASET AND IF IT IS BINARY

def check_df(df, target_variable):

  if df.empty :
      print('\n\n\n\nWe are sorry but we cannot procede with the execution: your file is empty ')
      raise SystemExit()


  result = any(item in df.columns for item in df.columns)

  if result==False:
      print('\n\n\n\nWe are sorry but we cannot procede with the execution: the target variable is not present in the dataset ')
      raise SystemExit()



  if len(df[target_variable].unique())>2:
      print('\n\n\n\nWe are sorry but we cannot procede with the execution: the target variable is not binary')
      raise SystemExit()  



#-------------------------------------------------------------------------------------------------------------
#DOWNLOAD FILE FROM DRIVE -- case: download from googleDrive

def download_from_googleDrive(dataset_name, is_excel, is_csv):

  import pandas as pd
  from google.colab import drive
  drive.mount('/content/drive', force_remount=True)

  print(dataset_name)


  if is_excel:

    try:

      #xls = pd.ExcelFile('/content/drive/My Drive/Elena/polimi/IIYear/tesi/analysis/tool/dataset_tool.xlsx')
      xls=pd.ExcelFile(dataset_name)
      df = pd.read_excel(xls)
      
      return df

    except:
      print('\n\n\n\nWe are sorry but we cannot procede with the execution: your file does not exist')
      raise SystemExit()


  if is_csv:

    try: 
      df=pd.read_csv(dataset_name)
      return df

    except:
      print('\n\n\n\nWe are sorry but we cannot procede with the execution: your file does not exist')
      raise SystemExit()
    
   

#---------------------------------------------------------------------------------------------------------------

#CHECK IF THE DATASET NAME IS AN EXCEL OF CSV FILE (CHECK EXTENTION) -- case: download from googleDrive



def load_dataset_from_name(dataset_name, target_variable):

    import re

    is_excel= re.search(r'\.xls[xlsx]?$', dataset_name)
    is_csv=re.search(r'\.csv?$', dataset_name)

    
    if ( not is_excel and not is_csv):

      print('\n\n\n\nWe are sorry but we cannot procede with the execution: your filename does not follow the specification (neither excel file nor csv file)')
      raise SystemExit()

    return download_from_googleDrive(dataset_name, is_excel, is_csv)

#-------------------------------------------------------------------------------------------------------------

 #DOWNLOAD FILE  -- case: download from github link

def download_from_github(dataset_name):

  import requests as rq
  from io import BytesIO

  
  try:

      xls=pd.ExcelFile(dataset_name)

      df = pd.read_excel(xls , engine='openpyxl')

      return df
      
  except:
      print('\n\n\n\nWe are sorry but we cannot procede with the execution: your file does not exist')
      raise SystemExit()

  

#----------------------------------------------------------------------------------------------------------------

#CHECK IF THE DATASET NAME IS AN EXCEL OF CSV FILE (CHECK EXTENTION) -- case: download from github link



def load_dataset_from_link(dataset_name, target_variable):

  import re

  is_link=re.search(r'/([https|http]?:\/\/[^ ]*)/', dataset_name)   # (http|ftp|https):\/\/([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:\/~+#-]*[\w@?^=%&\/~+#-])


  if ( is_link==False):

    print('\n\n\n\nWe are sorry but we cannot procede with the execution: your filename does not follow the specification (not a link)')
    raise SystemExit()
  
  return download_from_github(dataset_name, target_variable)

#--------------------------------------------------------------------------------------------------------------
#REQUEST dataset and target TO THE USER -- dataset name

def load():

    
    import pandas as pd

    print('Hello!\n\n\n\nThis application is designed to help you to discover, starting from a dataset (a set of columns with data) and a target variable to predict,\n which are the most significant'+
          ' variable and subsets determining the target variable.\n\n Let\'s start!!\n')

    dataset_name_type=0
  

    while True:

      name=input('Please type the name of the dataset_file or the github link:\n')

      dataset_name=str(name)

      name_type=input('Is this the name of dataset or a link to github?\n(please write N in case of the dataset name, L in case of the github link):\n')

      dataset_name_type=str(name_type)

      if ( dataset_name_type=='N' or dataset_name_type=='L'):
        break
      else:
        print('\n\nWe are sorry but we cannot procede with the execution: your input doesn\'t follow the instruction above!\n\n')



    variable=input('Please specify the name of the target variable present in the dataset:\n')

    target_variable=str(variable)

    print('\n\n\n\nThank you!\n')

    df=pd.DataFrame()

    if dataset_name_type=='N':
      df=load_dataset_from_name(dataset_name, target_variable)
    else:
      df=load_dataset_from_link(dataset_name, target_variable)


    check_df(df, target_variable)

    return df, target_variable


In [ ]:
df, target_variable=load()

Hello!



This application is designed to help you to discover, starting from a dataset (a set of columns with data) and a target variable to predict,
 which are the most significant variable and subsets determining the target variable.

 Let's start!!

Please type the name of the dataset_file or the github link:
/content/drive/My Drive/Elena/polimi/IIYear/tesi/analysis/tool/dataset_tool.xlsx
Is this the name of dataset or a link to github?
(please write N in case of the dataset name, L in case of the github link):
N
Please specify the name of the target variable present in the dataset:
Death




Thank you!

Mounted at /content/drive
/content/drive/My Drive/Elena/polimi/IIYear/tesi/analysis/tool/dataset_tool.xlsx


#ITERATE OVER ALL DATAFRAME COLUMNS AND CHECK FOR SIGNIFICANT CLUSTERS

In [2]:
def define_order(df):

  new_orders=dict()

  for col in df.columns:

    if df[col].dtype=='object':

      new_orders[col]=define_user_order(df[col], col)

  return new_orders


def define_user_order(lst, col):

  from sklearn import preprocessing
  import numpy as np

  print('\n\nPlease define the order of the values in the column '+col+'\nThe values in the column are:\n')

  for e in lst.unique():

    print(str(e))
  

  #taking in input the order

  while(True):
      i=1
      new_order=list()

      while i<=len(lst.unique()):

      
        new_order.append(input('\nImmetti il '+str(i)+'° valore:'))

        i=i+1

      exit=False

      #checking if the values in the list are correct
      if(sorted(lst.unique()) == sorted(new_order)):
        return new_order
      else:
        print('\n\nWe are sorry but we cannot procede with the execution: your input doesn\'t follow the instruction above!\n\n')




def use_user_order(lst, col, orders):

  from sklearn import preprocessing
  import numpy as np
  
  le = preprocessing.LabelEncoder()
  le = le.fit(lst)
  le.classes_ = np.array(orders[col])

  return le.transform(lst)

In [3]:
from pandas.core.indexing import length_of_indexer
#TRANSFORMING VARIABLES

def transform_dataset(X, target_variable, orders):

    import pandas as pd
    from statistics import median
    import numpy as np

    #X=X[['Result_viremia_Isample', 'Death', 'Sex']]

    #y[target_variable]=y[target_variable].replace({y[target_variable].unique()[0]: 0, y[target_variable].unique()[1]: 1})


    column_names=X.columns

    X_new=X.copy()

    # trasform categorical columns into numerical ones

  
    for col in column_names:

      if X[col].dtypes=='object':

        # take the order from the user
        X[col+'new']=use_user_order(X[col], col, orders)
        X_new[col]=X[col+'new']

        #X[col+'new']= pd.factorize(X[col], sort=True)[0]
        #X_new[col] = pd.factorize(X[col], sort=True)[0]

    

    # transform numerical columns in 2 bins

    for col in column_names:

      if X[col].dtypes!='object':

        X[col]=X[col].astype(int)
        
        lst=list()


        if(len((X[col].unique()))!=1):

            #define the median value
            median=np.median(X[col].unique())


            #define the 2 means

            sorted_arr = sorted(X[col].unique())
            median_index = len(X[col].unique())//2
            sub_list1, sub_list2 = sorted_arr[:median_index],sorted_arr[median_index:]
            mean1=np.mean(sub_list1)
            mean2=np.mean(sub_list2)


            for e in X[col]:
              if e<=median:
                lst.append(float(mean1))
              else:
                lst.append(float(mean2))

        

            X[col+'new']=lst
            X_new[col]=lst
        
        else:

          X[col+'new']=X[col]
          X_new[col]=X[col]

  

    return X,X_new

 

#ordine dell'utente

""" NO DUMMY ENCODING

    for col in column_names:

        #dummy encoding

        listt=[]

        # create dummy variables
        dummies= pd.get_dummies(df[col], prefix = col )

        for value in X[col].unique():
          listt.append(col+'_'+str(value))

        dummies = dummies[listt]
        df = pd.concat([df, dummies], axis = 1)
        df = df.drop(col, axis = 1)


"""


#df,df_new=transform_dataset(df, target_variable)



" NO DUMMY ENCODING\n\n    for col in column_names:\n\n        #dummy encoding\n\n        listt=[]\n\n        # create dummy variables\n        dummies= pd.get_dummies(df[col], prefix = col )\n\n        for value in X[col].unique():\n          listt.append(col+'_'+str(value))\n\n        dummies = dummies[listt]\n        df = pd.concat([df, dummies], axis = 1)\n        df = df.drop(col, axis = 1)\n\n\n"

In [ ]:
print(transform_dataset(df, 'Death', []))

['M' 'F']
int64
[77.8125 44.625 ]
int64
[2.5 9. ]
['Critical' 'Mild' 'Moderate' 'Severe']
int64
[ 5.         24.08333333]
int64
[1.5 3.5]
int64
[0.5 2.5]
int64
[1. 0.]
['first_wave' 'second_wave' 'alpha' 'beta+gamma' 'others' 'delta-delta+'
 'omicron']
['P' 'N']
int64
[77. 44.]
int64
[2. 9.]
int64
[ 5. 24.]
int64
[1. 3.]
int64
[0. 2.]
int64
[1. 0.]
(    Sex  Age_at_admission  CCI Disease_severity_at_Hospital_admission  \
0     M                69    3                               Critical   
1     M                51    1                                   Mild   
2     M                65    2                               Moderate   
3     M                78    4                               Moderate   
4     M                59    1                               Moderate   
..   ..               ...  ...                                    ...   
228   M                69    3                               Moderate   
229   F                83   10                                 S

In [ ]:
from google.colab import drive
import pandas as pd
drive.mount('/content/drive', force_remount=True)
xls = pd.ExcelFile('/content/drive/My Drive/Elena/polimi/IIYear/tesi/analysis/tool/dataset_tool.xlsx')
df = pd.read_excel(xls)

Mounted at /content/drive


In [ ]:
df.dtypes

Sex                                                object
Age_at_admission                                  float64
CCI                                               float64
Disease_severity_at_Hospital_admission             object
Days_from_symptoms_onset_to_Hospital_admission    float64
Period                                            float64
Doses_of_SARS_CoV_2_Vaccine                       float64
Death                                             float64
Variant_class                                      object
Result_viremia_Isample                             object
dtype: object

In [4]:
#COMPUTE ODD RATIO AND COEFFICIENTS RANGE

def compute_result(sig_clusters, target_variable):

    import numpy as np
    import pandas as pd

    if sig_clusters== -1:
      return -1


    #odd_ratios=dict()
    result=dict()
    df_result=pd.DataFrame()

    for k, v in sig_clusters.items():

      results_as_html = v.summary().tables[1].as_html()
      dff=pd.read_html(results_as_html, header=0, index_col=0)[0]

      #print(df)

      df_result['OR']= np.exp(dff.coef)
      df_result['lower CI'] = np.exp(dff['[0.025'])
      df_result['upper CI'] = np.exp(dff['0.975]'])
      df_result['coef']=dff['coef']
      df_result['std err']=dff['std err']
      df_result['z']=dff['z']
      df_result['P>|z|']=dff['P>|z|']
      df_result['[0.025']=dff['[0.025']
      df_result['0.975]']=dff['0.975]']


      result[k]=df_result

    return result 




def compute_odd_ratios(sig_clusters):

    import numpy as np
    import pandas as pd

    if sig_clusters== -1:
      return -1


    odd_ratios=dict()

    for k, v in sig_clusters.items():

      results_as_html = v.summary().tables[1].as_html()
      df=pd.read_html(results_as_html, header=0, index_col=0)[0]
    
      odd_ratios[k]=float(np.exp(df.coef))

     
    print(odd_ratios)

    return odd_ratios





#-----------------------------------------------------------------------------------------------------------------------------
  #MULTIVARIATE LOGISTIC REGRESSION WITH SIGNIFICANT COLUMNS

def multivariate_logistic_regression(sig_columns,X,y):

    from sklearn import datasets, linear_model
    from sklearn.linear_model import LinearRegression
    from sklearn.model_selection import train_test_split
    import statsmodels.api as sm
    from imblearn.over_sampling import RandomOverSampler  
    import numpy as np
    import itertools

    sig_clusters=dict({})

    for L in range(len(sig_columns) + 1):
        for subset in itertools.combinations(sig_columns, L):

          if(len(subset)>0):
            #oversampling and splitting in test and train

            os = RandomOverSampler(random_state=0)
            X_train, X_test, y_train, y_test = train_test_split(X[list(subset)], y, test_size=0.3, random_state=0)
            columns = X_train.columns


            if ((any(y_train[target_variable] == 0)) and any(y_train[target_variable] == 1)) :
              os_data_X, os_data_y = os.fit_resample(X_train, y_train)
            
              #logistic regression
              log=sm.Logit(os_data_y,os_data_X)
              #log=sm.Logit(y_train, X_train)


              result=log.fit(method='bfgs')
              #print(result.summary())

              p_values = result.summary2().tables[1]['P>|z|']


              #check statistical significance

              if all(item<0.05 for item in p_values):

                sig_clusters[subset]=result

    return sig_clusters

#-------------------------------------------------------------------------------------------------------------------


def univariate_logistic_regression(X,y):


    import itertools
    from sklearn import datasets, linear_model
    from sklearn.linear_model import LinearRegression
    from sklearn.model_selection import train_test_split
    import statsmodels.api as sm
    from imblearn.over_sampling import SMOTE  
    from imblearn.over_sampling import RandomOverSampler
    import numpy as np

    if X.empty or X.shape[0]<=1:
      return -1

    column_names=X.columns
    
    sig_columns=list()


    #UNIVARIATE LOGISTIC REGRESSION ON SINGLE COLUMNS

    for col in column_names:

            #oversampling and splitting in test and train

            
            os = RandomOverSampler(random_state=0)
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)


            
            if ((any(y_train[target_variable] == 0)) and any(y_train[target_variable] == 1)) :

              os_data_X, os_data_y = os.fit_resample(X_train, y_train)
              os_data_X=os_data_X[col]
            
              #logistic regression
              log=sm.Logit(os_data_y,os_data_X)
              #log=sm.Logit(y_train,X_train[col])

          

              result=log.fit(method='bfgs')
              #print(result.summary())


              p_values = result.summary2().tables[1]['P>|z|']
              coeff= result.summary2().tables[1]['Coef.']

              odd_ratio= np.exp(coeff)


              if(p_values[0]<=0.05 and odd_ratio[0]>=1):

                sig_columns.append(col)

    print(len(sig_columns))

    #call multivariate function
    return multivariate_logistic_regression(sig_columns,X,y)


#---------------------------------------------------------------------------------------------------------------------


#ITERATE OVER ALL DATAFRAME COLUMNS AND CHECK FOR SIGNIFICANT CLUSTERS

def iterate_over_columns(X,y, target_variable):

    import itertools
    from imblearn.over_sampling import SMOTE       
    import numpy as np
    from sklearn import datasets, linear_model
    from sklearn.linear_model import LinearRegression
    from sklearn.model_selection import train_test_split
    import statsmodels.api as sm
    from scipy import stats


    #call univariate logistic regression

    sig_clusters=univariate_logistic_regression(X,y)
        
    #COMPUTE P_VALUE ODD_RATIO AND CI FOR ALL SUBSETS

    result=compute_result(sig_clusters, target_variable)

    odd_ratios=compute_odd_ratios(sig_clusters)
   
    return result, odd_ratios, target_variable




In [ ]:

X = df_new.loc[:, df_new.columns != target_variable] # data in input
y = df_new.loc[:, df_new.columns == target_variable] # data to predict

sig_clusters, target_variable=iterate_over_columns(X,y,target_variable)

NameError: ignored

In [5]:
def compute_subsets_of_column(df, col):

  import itertools

   
  temp=list()

  for L in range(len(df[col].unique()) + 1):
        for subset in itertools.combinations(df[col].unique(), L):

          temp.append((col,subset))

  return temp


def create_subsets(combinations, df, target_variable):

  import pandas as pd

  subsets_df=dict()


  for comb in combinations:

      temp_string=' '
      df1=df.copy()
      df2=pd.DataFrame()
      consider=True

      for el in list(comb): # month - june, july , day 1, 2

        col_name=el[0]
        temp_string=temp_string+'    '+col_name+'='+ str(el[1])

        if not df2.empty:
          df1=df2
          df2=pd.DataFrame()

        if(len(el[1])!=0):
          
        

          for ell in el[1]: #june #july


              df1=df1[df1[col_name]==ell] #filter

              df2=df2.append(df1)

          if (len(el[1])==1):
            del df2[col_name] #drop column
        
        else:
          
          consider=False

       
        if consider:
          subsets_df[temp_string]=df2


  #delete columns with only one value

  """
  for sub in subsets_df.values():

    for col in sub.columns:

        if col!=target_variable:

          if len(set(sub[col]))==1:

            del sub[col]

  """

  #print(subsets_df)

  return subsets_df


In [6]:
def iterate_over_rows(X,target_variable):

  import pandas as pd

  #create list with column values subsets

  col_values=list()


  col_names=X.columns

  for col in col_names:

    if(col!=target_variable):
      temp=compute_subsets_of_column(X, col)

      col_values.append(temp)


  print(col_values)


  #define order inside columns

  orders=define_order(X)




  #all possible iterations between columns

  import itertools

  combinations= list(itertools.product(*col_values))

  #print(combinations)




  #filter dataframe to make the subsets

  subsets_df=dict()

  subsets_df=create_subsets(combinations, X, target_variable)


  
  #call iteration over columns

  result=dict()
  odd=dict()

  i=0

  for key, value in subsets_df.items():

        
        df,df_new=transform_dataset(value, target_variable, orders)

        X,y=create_X_y(df_new,target_variable)

        #print('\n\n\ncase: '+ str(key))
        sig_clusters, odd_ratios, target_variable=iterate_over_columns(X, y, target_variable)



        if(sig_clusters!=-1 and  len(sig_clusters)!=0):



          #if sig_clusters:

          for k,v in sig_clusters.items():

            result[key]=v

          for k,v in odd_ratios.items():

            odd[key]=v
            


  #order result
  result=order_result(result, odd)


  #show result
  show_result(result, target_variable)


  return result
  

In [8]:
def create_X_y(df, target_variable):

    X = df.loc[:, df.columns != target_variable] # data in input
    y = df.loc[:, df.columns == target_variable] # data to predict

    
    return X,y

In [9]:
def order_result(result, odd):

  return dict(sorted(result.items(), key=lambda kv: odd[kv[0]]))




def show_result(result, target_variable):

  print('\n\nBelow the list of significant subsets and columns for the target variable '+target_variable+':\n')


  for k,v in result.items():
  
    print('\nthe subset is defined as:')
    print(k)
    print('\nthose are the significant columns with their relative p_value, OR and CI:\n')
    print(v)
    print('\n')
    print('----------------------')



In [ ]:


df, target_variable=load()

df=df[['Death','Sex', 'Period', 'Age_at_admission', 'CCI', 'Days_from_symptoms_onset_to_Hospital_admission', 'Doses_of_SARS_CoV_2_Vaccine', 'Result_viremia_Isample']]
df=df[(df['Period']==3) | (df['Period']==4)]

df=df[['Death','Sex', 'Age_at_admission', 'CCI', 'Days_from_symptoms_onset_to_Hospital_admission', 'Doses_of_SARS_CoV_2_Vaccine', 'Result_viremia_Isample']]

print(df)
result=iterate_over_rows(df,target_variable)

#ordina per odd ratio

#adjusted odd ratio -- con multipli variabili odd ratio <1-- o >1---
#tutte variabli con odd ratio ok e fare multivarita e poi calcolo adjusted odd ratio

Hello!



This application is designed to help you to discover, starting from a dataset (a set of columns with data) and a target variable to predict,
 which are the most significant variable and subsets determining the target variable.

 Let's start!!

Please type the name of the dataset_file or the github link:
/content/drive/My Drive/Elena/polimi/IIYear/tesi/analysis/tool/dataset_tool.xlsx
Is this the name of dataset or a link to github?
(please write N in case of the dataset name, L in case of the github link):
N
Please specify the name of the target variable present in the dataset:
Death




Thank you!

Mounted at /content/drive
/content/drive/My Drive/Elena/polimi/IIYear/tesi/analysis/tool/dataset_tool.xlsx
     Death Sex  Age_at_admission   CCI  \
34     0.0   F         49.242984   0.0   
35     0.0   F         81.409993   5.0   
36     0.0   M         63.096509   2.0   
37     1.0   F         92.407940   6.0   
38     0.0   M         52.147844   3.0   
..     ...  ..           